# Data Description & Methodology with Results

In [146]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans, DBSCAN

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

In [147]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


### Load and explore the data

In [148]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

#### Tranform the data into a *pandas* dataframe

In [149]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude


In [150]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### This is only making sure we have correct number of boroughs and neighbourhoods.

In [151]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.

In [152]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [153]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Now, we will extract data of Manhattan and Bronx.

Firstly, we extract the data from Manhattan.

In [154]:
# Manhattan
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [155]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
m_location = geolocator.geocode(address)
m_latitude = m_location.latitude
m_longitude = m_location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(m_latitude, m_longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [156]:
# Bronx
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [157]:
address = ' Bronx, NY, USA'

geolocator = Nominatim(user_agent="ny_explorer")
b_location = geolocator.geocode(address)
b_latitude = b_location.latitude
b_longitude = b_location.longitude
print('The geograpical coordinate of The Bronx are {}, {}.'.format(b_latitude, b_longitude))

The geograpical coordinate of The Bronx are 40.8466508, -73.8785937.


#### Next, define Foursquare Credentials and Version

In [158]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: *****
CLIENT_SECRET:*****


#### Let's make an example: we explore the first neighborhood in Manhattan and get the neighborhood's latitude and longitude values.

In [159]:
# Get the neighborhood's name.
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

In [160]:
neighborhood_latitude_m = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude_m = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name_m = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name_m, 
                                                               neighborhood_latitude_m, 
                                                               neighborhood_longitude_m))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [161]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude_m, neighborhood_longitude_m, VERSION, radius, LIMIT)
print('URL existed:', url != "")

URL existed: True


Send the GET request and examine the resutls

In [162]:
results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [163]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [164]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [165]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Manhattan & Bronx
### 2.1 Manhattan

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [166]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [167]:
# type your answer here
LIMIT = 100
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of dataframe and how many venues were returned for each neighborhood.

In [168]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3309, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [169]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,97,97,97,97,97,97
Carnegie Hill,100,100,100,100,100,100
Central Harlem,46,46,46,46,46,46
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,40,40,40,40,40,40
East Village,100,100,100,100,100,100


In [170]:
print('There are {} uniques categories in Manhattan.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 337 uniques categories in Manhattan.


### 2.2 Bronx

In [171]:
# type your answer here
LIMIT = 100
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [172]:
print(bronx_venues.shape)
bronx_venues.head()

(1225, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station
4,Wakefield,40.894705,-73.847201,SUBWAY,40.890468,-73.849152,Sandwich Place


In [173]:
bronx_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,37,37,37,37,37,37
Baychester,18,18,18,18,18,18
Bedford Park,36,36,36,36,36,36
Belmont,99,99,99,99,99,99
Bronxdale,12,12,12,12,12,12
Castle Hill,9,9,9,9,9,9
City Island,27,27,27,27,27,27
Claremont Village,18,18,18,18,18,18
Clason Point,9,9,9,9,9,9


In [174]:
print('There are {} uniques categories in Bronx.'.format(len(bronx_venues['Venue Category'].unique())))

There are 161 uniques categories in Bronx.


## 3. Analyze Each Neighborhood
### 3.1 Manhattan

In [175]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shoppin

In [176]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Columns: 338 entries, Neighborhood to Yoga Studio
dtypes: float64(337), object(1)
memory usage: 105.7+ KB


In [177]:
# Let's confirm the new size.
manhattan_grouped.shape

(40, 338)

In [178]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0           Park  0.07
1    Coffee Shop  0.06
2          Hotel  0.05
3            Gym  0.04
4  Memorial Site  0.03


----Carnegie Hill----
            venue  freq
0     Coffee Shop  0.07
1     Pizza Place  0.05
2  Cosmetics Shop  0.04
3     Yoga Studio  0.03
4          Bakery  0.03


----Central Harlem----
                 venue  freq
0   African Restaurant  0.07
1        Deli / Bodega  0.04
2                  Bar  0.04
3  Fried Chicken Joint  0.04
4    French Restaurant  0.04


----Chelsea----
                venue  freq
0         Coffee Shop  0.06
1              Bakery  0.05
2  Italian Restaurant  0.04
3      Ice Cream Shop  0.04
4             Theater  0.03


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.10
1           Cocktail Bar  0.04
2  Vietnamese Restaurant  0.04
3    American Restaurant  0.04
4           Optical Shop  0.03


----Civic Center----
                  venue  freq
0  Gym / Fitness Center 

#### Once getting results above, we put that into a *pandas* dataframe

In [179]:
# Write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [180]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_m = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_m['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted_m.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_m.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Wine Shop,Women's Store,Boat or Ferry,Italian Restaurant,BBQ Joint
1,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Bakery,Wine Shop,Bookstore,Café,French Restaurant,Grocery Store,Gym
2,Central Harlem,African Restaurant,Deli / Bodega,French Restaurant,Cosmetics Shop,Seafood Restaurant,Fried Chicken Joint,Chinese Restaurant,Bar,American Restaurant,Event Space
3,Chelsea,Coffee Shop,Bakery,Ice Cream Shop,Italian Restaurant,Nightclub,Wine Shop,American Restaurant,Hotel,Theater,Men's Store
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Spa,Bakery,Hotpot Restaurant,Salon / Barbershop,Noodle House


### 3.2 Bronx
Repeat the steps above on Bronx data.

In [181]:
# one hot encoding
bronx_onehot = pd.get_dummies(bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Neighborhood'] = bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nightclub,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [182]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Columns: 162 entries, Neighborhood to Women's Store
dtypes: float64(161), object(1)
memory usage: 65.9+ KB


In [183]:
# Let's confirm the new size.
bronx_grouped.shape

(52, 162)

In [184]:
num_top_venues = 5

for hood in bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Pizza Place  0.16
1       Deli / Bodega  0.08
2         Supermarket  0.08
3  Chinese Restaurant  0.05
4    Department Store  0.05


----Baychester----
                 venue  freq
0           Donut Shop  0.11
1       Discount Store  0.06
2                 Bank  0.06
3   Spanish Restaurant  0.06
4  Fried Chicken Joint  0.06


----Bedford Park----
                venue  freq
0               Diner  0.11
1  Chinese Restaurant  0.11
2         Supermarket  0.08
3         Pizza Place  0.08
4         Bus Station  0.08


----Belmont----
                venue  freq
0  Italian Restaurant  0.18
1         Pizza Place  0.09
2       Deli / Bodega  0.08
3              Bakery  0.05
4          Donut Shop  0.03


----Bronxdale----
                   venue  freq
0     Italian Restaurant  0.08
1  Performing Arts Venue  0.08
2            Supermarket  0.08
3         Breakfast Spot  0.08
4     Mexican Restaurant  0.08


----Castle Hill----
            ven

In [185]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted_b = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted_b['Neighborhood'] = bronx_grouped['Neighborhood']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted_b.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted_b.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega,Chinese Restaurant,Department Store,Discount Store,Construction & Landscaping,Donut Shop,Dessert Shop,Martial Arts Dojo
1,Baychester,Donut Shop,Convenience Store,Bank,Pet Store,Discount Store,Sandwich Place,Electronics Store,Fast Food Restaurant,Mexican Restaurant,Men's Store
2,Bedford Park,Diner,Chinese Restaurant,Supermarket,Pizza Place,Bus Station,Deli / Bodega,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Park
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Liquor Store,Mexican Restaurant,Donut Shop,Dessert Shop,Bank,Shoe Store
4,Bronxdale,Spanish Restaurant,Bank,Performing Arts Venue,Paper / Office Supplies Store,Chinese Restaurant,Eastern European Restaurant,Mexican Restaurant,Breakfast Spot,Supermarket,Pizza Place


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.
### 4.1 Manhattan

In [186]:
# glimpse the dataframe which is for clustering.
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shoppin

In [187]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 4, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [188]:
# add clustering labels
neighborhoods_venues_sorted_m.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted_m.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Coffee Shop,Yoga Studio,Diner,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Tennis Stadium,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Spa,Bakery,Hotpot Restaurant,Salon / Barbershop,Noodle House
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Grocery Store,Mobile Phone Shop,Coffee Shop,Deli / Bodega,Mexican Restaurant,Latin American Restaurant,New American Restaurant,Park
3,Manhattan,Inwood,40.867684,-73.921210,4,Café,Mexican Restaurant,Restaurant,Lounge,Frozen Yogurt Shop,Spanish Restaurant,Park,Chinese Restaurant,Deli / Bodega,Bakery
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Yoga Studio,Indian Restaurant,Sushi Restaurant,Park,Cocktail Bar


Finally, let's visualize the resulting clusters

In [189]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 4.2 Bronx

In [190]:
# glimpse the dataframe which is for clustering.
bronx_grouped.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Boat or Ferry,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Donut Shop,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,High School,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moving Target,Music Store,Music Venue,Nightclub,Outdoor Sculpture,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tapas Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027027,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.027027,0.0,0.0,0.0,0.0,0.0,0.0,0.027027,0.0,0.000000,0.0,0.0,0.027027,0.000000,0.054054,0.0,0.000000,0.0,0.027027,0.000000,0.0,0.0,0.081081,0.054054,0.027027,0.000000,0.027027,0.0,0.0,0.027027,0.000000,0.027027,0.0,0.027027,0.0,0.000000,0.0,0.027027,0.000000,0.000000,0.0,0.027027,0.0,0.0,0.027027,0.0,0.000000,0.0,0.027027,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.027027,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.027027,0.027027,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.027027,0.000000,0.162162,0.0,0.027027,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.027027,0.027027,0.0,0.0,0.0,0.081081,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Baychester,0.0,0.0,0.000000,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.055556,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.055556,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.055556,0.0,0.0,0.111111,0.000000,0.055556,0.0,0.055556,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.055556,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.055556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.055556,0.0,0.055556,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0

In [191]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 2, 0, 1], dtype=int32)

In [192]:
# add clustering labels
neighborhoods_venues_sorted_b.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(neighborhoods_venues_sorted_b.set_index('Neighborhood'), on='Neighborhood')

bronx_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,2,Ice Cream Shop,Sandwich Place,Gas Station,Caribbean Restaurant,Laundromat,Donut Shop,Pharmacy,Dessert Shop,Discount Store,Distillery
1,Bronx,Co-op City,40.874294,-73.829939,1,Bus Station,Accessories Store,Gift Shop,Pharmacy,Park,Discount Store,Restaurant,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Bowling Alley,Pizza Place,Platform,Convenience Store,Donut Shop,Chinese Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,4,Plaza,Music Venue,Bus Station,River,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Plaza,Gym,Bus Station,Moving Target,Bank,Food Truck,Home Service,Health & Beauty Service,Donut Shop


In [193]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
Now, we will examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.
### 5.1 Manhattan

#### Cluster 1

In [194]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Spa,Bakery,Hotpot Restaurant,Salon / Barbershop,Noodle House
6,Central Harlem,African Restaurant,Deli / Bodega,French Restaurant,Cosmetics Shop,Seafood Restaurant,Fried Chicken Joint,Chinese Restaurant,Bar,American Restaurant,Event Space
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,Juice Bar,Coffee Shop,Gym / Fitness Center,American Restaurant,Pizza Place,Hotel
13,Lincoln Square,Theater,Italian Restaurant,Café,Concert Hall,Plaza,Performing Arts Venue,Gym / Fitness Center,French Restaurant,Indie Movie Theater,Clothing Store
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,American Restaurant,Wine Shop,Gym,Hotel,Spa,Sandwich Place
15,Midtown,Hotel,Coffee Shop,Theater,Clothing Store,Sporting Goods Shop,Bakery,Steakhouse,Japanese Restaurant,Bookstore,Gym
17,Chelsea,Coffee Shop,Bakery,Ice Cream Shop,Italian Restaurant,Nightclub,Wine Shop,American Restaurant,Hotel,Theater,Men's Store
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,Seafood Restaurant,Indian Restaurant,French Restaurant,Gym,Chinese Restaurant,Dessert Shop
21,Tribeca,American Restaurant,Italian Restaurant,Park,Café,Spa,Men's Store,Coffee Shop,Greek Restaurant,Wine Bar,Wine Shop
22,Little Italy,Bakery,Café,Bubble Tea Shop,Italian Restaurant,Cocktail Bar,Sandwich Place,Salon / Barbershop,Mediterranean Restaurant,Japanese Restaurant,Spa


#### Cluster 2

In [195]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Wine Shop,Sushi Restaurant,Japanese Restaurant,Diner
10,Lenox Hill,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Cocktail Bar,Burger Joint,Café,Gym,Gym / Fitness Center,Deli / Bodega
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Indian Restaurant,Bakery,Café,Mediterranean Restaurant,Restaurant,Pub
16,Murray Hill,Coffee Shop,Sandwich Place,American Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bar,Gym / Fitness Center,Restaurant,Cuban Restaurant
19,East Village,Bar,Ice Cream Shop,Wine Bar,Pizza Place,Mexican Restaurant,Chinese Restaurant,Cocktail Bar,Speakeasy,Japanese Restaurant,Coffee Shop
25,Manhattan Valley,Bar,Indian Restaurant,Coffee Shop,Yoga Studio,Playground,Pizza Place,Mexican Restaurant,Thai Restaurant,Japanese Restaurant,Hawaiian Restaurant
27,Gramercy,Italian Restaurant,Bar,Pizza Place,Mexican Restaurant,Bagel Shop,Cocktail Bar,Playground,Thai Restaurant,Grocery Store,Thrift / Vintage Store
29,Financial District,Coffee Shop,American Restaurant,Bar,Food Truck,Hotel,Pizza Place,Gym,Italian Restaurant,Cocktail Bar,Gym / Fitness Center
30,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Bakery,Wine Shop,Bookstore,Café,French Restaurant,Grocery Store,Gym
31,Noho,Italian Restaurant,French Restaurant,Cocktail Bar,Coffee Shop,American Restaurant,Grocery Store,Rock Club,Pizza Place,Mexican Restaurant,Hotel


#### Cluster 3

In [196]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Playground,Harbor / Marina,Coffee Shop,Cocktail Bar,Gas Station,Farmers Market,Heliport,Pet Service


#### Cluster 4

In [197]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Midtown South,Korean Restaurant,Hotel Bar,Japanese Restaurant,Coffee Shop,Hotel,Dessert Shop,Cocktail Bar,American Restaurant,Cosmetics Shop,Fried Chicken Joint


#### Cluster 5

In [198]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Yoga Studio,Diner,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Tennis Stadium,Seafood Restaurant
2,Washington Heights,Café,Bakery,Grocery Store,Mobile Phone Shop,Coffee Shop,Deli / Bodega,Mexican Restaurant,Latin American Restaurant,New American Restaurant,Park
3,Inwood,Café,Mexican Restaurant,Restaurant,Lounge,Frozen Yogurt Shop,Spanish Restaurant,Park,Chinese Restaurant,Deli / Bodega,Bakery
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Yoga Studio,Indian Restaurant,Sushi Restaurant,Park,Cocktail Bar
5,Manhattanville,Coffee Shop,Deli / Bodega,Italian Restaurant,Park,Chinese Restaurant,Mexican Restaurant,Seafood Restaurant,Falafel Restaurant,Liquor Store,Lounge
7,East Harlem,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,Bakery,Deli / Bodega,Moving Target,Grocery Store,Cocktail Bar,Sandwich Place,Steakhouse
11,Roosevelt Island,Park,Sandwich Place,Dog Run,Gym,Greek Restaurant,Scenic Lookout,Liquor Store,School,Outdoors & Recreation,Coffee Shop
20,Lower East Side,Art Gallery,Pizza Place,Café,Coffee Shop,Shoe Store,Japanese Restaurant,Ramen Restaurant,Bakery,Chinese Restaurant,Cocktail Bar
26,Morningside Heights,Park,Bookstore,Coffee Shop,American Restaurant,Sandwich Place,Burger Joint,Deli / Bodega,Food Truck,Ice Cream Shop,Greek Restaurant
36,Tudor City,Café,Mexican Restaurant,Park,Greek Restaurant,Deli / Bodega,Pizza Place,Diner,Coffee Shop,Spanish Restaurant,Garden


### 5.2 Bronx

#### Cluster 1

In [199]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 0, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Riverdale,Park,Plaza,Gym,Bus Station,Moving Target,Bank,Food Truck,Home Service,Health & Beauty Service,Donut Shop
17,West Farms,Bus Station,Park,Donut Shop,Metro Station,Scenic Lookout,Sandwich Place,Lounge,Supermarket,Basketball Court,Bus Line
26,Clason Point,Park,Grocery Store,Boat or Ferry,Bus Stop,South American Restaurant,Pool,Diner,Farmers Market,Flower Shop,Fish Market
34,Spuyten Duyvil,Tennis Stadium,Pharmacy,Park,Intersection,Bus Line,Bus Stop,Scenic Lookout,Thai Restaurant,Bank,Tennis Court


#### Cluster 2

In [200]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 1, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bus Station,Accessories Store,Gift Shop,Pharmacy,Park,Discount Store,Restaurant,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Bowling Alley,Pizza Place,Platform,Convenience Store,Donut Shop,Chinese Restaurant
5,Kingsbridge,Pizza Place,Sandwich Place,Bar,Supermarket,Latin American Restaurant,Deli / Bodega,Mexican Restaurant,Discount Store,Pharmacy,Spanish Restaurant
6,Woodlawn,Pub,Deli / Bodega,Pizza Place,Playground,Cosmetics Shop,Supermarket,Food & Drink Shop,Rental Car Location,Bar,Moving Target
7,Norwood,Pizza Place,Park,Bank,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Mexican Restaurant,Sandwich Place,Caribbean Restaurant,Spanish Restaurant
8,Williamsbridge,Spa,Soup Place,Bar,Nightclub,Caribbean Restaurant,Farmers Market,Food,Flower Shop,Fish Market,Fish & Chips Shop
9,Baychester,Donut Shop,Convenience Store,Bank,Pet Store,Discount Store,Sandwich Place,Electronics Store,Fast Food Restaurant,Mexican Restaurant,Men's Store
10,Pelham Parkway,Pizza Place,Chinese Restaurant,Italian Restaurant,Home Service,Sushi Restaurant,Plaza,Coffee Shop,Donut Shop,Sandwich Place,Café
11,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Liquor Store,Bar,Diner,Deli / Bodega,Pizza Place,History Museum,Music Venue
12,Bedford Park,Diner,Chinese Restaurant,Supermarket,Pizza Place,Bus Station,Deli / Bodega,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Park


#### Cluster 3

In [201]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 2, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Ice Cream Shop,Sandwich Place,Gas Station,Caribbean Restaurant,Laundromat,Donut Shop,Pharmacy,Dessert Shop,Discount Store,Distillery
25,Soundview,Chinese Restaurant,Grocery Store,Latin American Restaurant,Breakfast Spot,Bus Station,Fried Chicken Joint,Bus Stop,Burger Joint,Pharmacy,Discount Store
40,Olinville,Supermarket,Caribbean Restaurant,Liquor Store,Chinese Restaurant,Fried Chicken Joint,Deli / Bodega,Basketball Court,Laundromat,Food,Eastern European Restaurant
42,Concourse,Grocery Store,Pizza Place,Supermarket,Chinese Restaurant,Bakery,Convenience Store,Metro Station,Sandwich Place,Caribbean Restaurant,Spanish Restaurant
44,Edenwald,Grocery Store,Supermarket,Fish Market,Chinese Restaurant,Gas Station,Women's Store,Food,Flower Shop,Fish & Chips Shop,Fast Food Restaurant
45,Claremont Village,Bus Station,Chinese Restaurant,Pizza Place,Grocery Store,Gym,Deli / Bodega,Supermarket,Caribbean Restaurant,Gift Shop,Liquor Store
48,Mount Hope,Grocery Store,Video Game Store,Supermarket,Sandwich Place,Mexican Restaurant,Donut Shop,Ice Cream Shop,Deli / Bodega,Spanish Restaurant,Discount Store


#### Cluster 4

In [202]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 3, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Country Club,Sandwich Place,Playground,Spa,Italian Restaurant,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 5

In [203]:
bronx_merged.loc[bronx_merged['Cluster Labels'] == 4, bronx_merged.columns[[1] + list(range(5, bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,Music Venue,Bus Station,River,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


## 5. DBSCAN-Clustering Neighborhoods
Run *DBSCAN* to cluster the neighborhood into 5 clusters. Here, we will cluster the neighbourhoods by their latitude and longitude.
### 5.1 Manhattan

In [204]:
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler 


In [205]:
manhattan_data_copy = manhattan_data.copy()
manhattan_data_copy.drop('Borough', axis=1, inplace=True)
manhattan_data_copy.head()

,Neighborhood,Latitude,Longitude
0,Marble Hill,40.876551,-73.910660
1,Chinatown,40.715618,-73.994279
2,Washington Heights,40.851903,-73.936900
3,Inwood,40.867684,-73.921210
4,Hamilton Heights,40.823604,-73.949688


In [217]:
# Preprocess the location data
Clust_manhattan = manhattan_data_copy[['Latitude', 'Longitude']]
Clust_manhattan = StandardScaler().fit_transform(Clust_manhattan)

# Compute DBSCAN
db = DBSCAN(eps=0.5, min_samples=3).fit(Clust_manhattan)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
manhattan_data_copy["Clus_Dbscan"]=labels
manhattan_merged_dbscan = pd.concat([manhattan_merged, manhattan_data_copy[['Clus_Dbscan']]], axis=1, join='inner')

# Check the number of DBSCAN clusters.
num_clus = len(set(labels))
print(set(labels))

{0, 1, -1}


In [218]:
# In order to prevent error, all DBSCAN cluster labels will add 1 to make them non-negative.
manhattan_merged_dbscan['Clus_Dbscan'] = manhattan_merged_dbscan['Clus_Dbscan'] + 1
# Check the resulting dataframe.
manhattan_merged_dbscan.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Sandwich Place,Coffee Shop,Yoga Studio,Diner,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Tennis Stadium,Seafood Restaurant,0
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Spa,Bakery,Hotpot Restaurant,Salon / Barbershop,Noodle House,1
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Grocery Store,Mobile Phone Shop,Coffee Shop,Deli / Bodega,Mexican Restaurant,Latin American Restaurant,New American Restaurant,Park,0
3,Manhattan,Inwood,40.867684,-73.921210,4,Café,Mexican Restaurant,Restaurant,Lounge,Frozen Yogurt Shop,Spanish Restaurant,Park,Chinese Restaurant,Deli / Bodega,Bakery,0
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Yoga Studio,Indian Restaurant,Sushi Restaurant,Park,Cocktail Bar,2


In [219]:
# Cluster 1
manhattan_merged_dbscan.loc[manhattan_merged_dbscan['Clus_Dbscan'] == 1, manhattan_merged_dbscan.columns[[1] + list(range(5, manhattan_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Optical Shop,Spa,Bakery,Hotpot Restaurant,Salon / Barbershop,Noodle House,1
7,East Harlem,Mexican Restaurant,Thai Restaurant,Latin American Restaurant,Bakery,Deli / Bodega,Moving Target,Grocery Store,Cocktail Bar,Sandwich Place,Steakhouse,1
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,Juice Bar,Coffee Shop,Gym / Fitness Center,American Restaurant,Pizza Place,Hotel,1
9,Yorkville,Italian Restaurant,Coffee Shop,Bar,Gym,Pizza Place,Deli / Bodega,Wine Shop,Sushi Restaurant,Japanese Restaurant,Diner,1
10,Lenox Hill,Coffee Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Cocktail Bar,Burger Joint,Café,Gym,Gym / Fitness Center,Deli / Bodega,1
11,Roosevelt Island,Park,Sandwich Place,Dog Run,Gym,Greek Restaurant,Scenic Lookout,Liquor Store,School,Outdoors & Recreation,Coffee Shop,1
12,Upper West Side,Italian Restaurant,Wine Bar,Bar,Coffee Shop,Indian Restaurant,Bakery,Café,Mediterranean Restaurant,Restaurant,Pub,1
13,Lincoln Square,Theater,Italian Restaurant,Café,Concert Hall,Plaza,Performing Arts Venue,Gym / Fitness Center,French Restaurant,Indie Movie Theater,Clothing Store,1
14,Clinton,Theater,Italian Restaurant,Gym / Fitness Center,Coffee Shop,American Restaurant,Wine Shop,Gym,Hotel,Spa,Sandwich Place,1
15,Midtown,Hotel,Coffee Shop,Theater,Clothing Store,Sporting Goods Shop,Bakery,Steakhouse,Japanese Restaurant,Bookstore,Gym,1


In [220]:
# Cluster 2
manhattan_merged_dbscan.loc[manhattan_merged_dbscan['Clus_Dbscan'] == 2, manhattan_merged_dbscan.columns[[1] + list(range(5, manhattan_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Deli / Bodega,Mexican Restaurant,Yoga Studio,Indian Restaurant,Sushi Restaurant,Park,Cocktail Bar,2
5,Manhattanville,Coffee Shop,Deli / Bodega,Italian Restaurant,Park,Chinese Restaurant,Mexican Restaurant,Seafood Restaurant,Falafel Restaurant,Liquor Store,Lounge,2
6,Central Harlem,African Restaurant,Deli / Bodega,French Restaurant,Cosmetics Shop,Seafood Restaurant,Fried Chicken Joint,Chinese Restaurant,Bar,American Restaurant,Event Space,2
25,Manhattan Valley,Bar,Indian Restaurant,Coffee Shop,Yoga Studio,Playground,Pizza Place,Mexican Restaurant,Thai Restaurant,Japanese Restaurant,Hawaiian Restaurant,2
26,Morningside Heights,Park,Bookstore,Coffee Shop,American Restaurant,Sandwich Place,Burger Joint,Deli / Bodega,Food Truck,Ice Cream Shop,Greek Restaurant,2


In [221]:
# Noise
manhattan_merged_dbscan.loc[manhattan_merged_dbscan['Clus_Dbscan'] == 0, manhattan_merged_dbscan.columns[[1] + list(range(5, manhattan_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
0,Marble Hill,Sandwich Place,Coffee Shop,Yoga Studio,Diner,Steakhouse,Miscellaneous Shop,Supplement Shop,Shopping Mall,Tennis Stadium,Seafood Restaurant,0
2,Washington Heights,Café,Bakery,Grocery Store,Mobile Phone Shop,Coffee Shop,Deli / Bodega,Mexican Restaurant,Latin American Restaurant,New American Restaurant,Park,0
3,Inwood,Café,Mexican Restaurant,Restaurant,Lounge,Frozen Yogurt Shop,Spanish Restaurant,Park,Chinese Restaurant,Deli / Bodega,Bakery,0


In [224]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clus)
ys = [i + x + (i*x)**2 for i in range(num_clus)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged_dbscan['Latitude'], manhattan_merged_dbscan['Longitude'], manhattan_merged_dbscan['Neighborhood'], manhattan_merged_dbscan['Clus_Dbscan']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5.2 Bronx

In [223]:
bronx_data_copy = bronx_data.copy()
bronx_data_copy.drop('Borough', axis=1, inplace=True)
bronx_data_copy.head()

,Neighborhood,Latitude,Longitude
0,Wakefield,40.894705,-73.847201
1,Co-op City,40.874294,-73.829939
2,Eastchester,40.887556,-73.827806
3,Fieldston,40.895437,-73.905643
4,Riverdale,40.890834,-73.912585


In [225]:
# Preprocess the location data
Clust_bronx = bronx_data_copy[['Latitude', 'Longitude']]
Clust_bronx = StandardScaler().fit_transform(Clust_bronx)

# Compute DBSCAN
db = DBSCAN(eps=0.5, min_samples=3).fit(Clust_bronx)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
bronx_data_copy["Clus_Dbscan"]=labels
bronx_merged_dbscan = pd.concat([bronx_merged, bronx_data_copy[['Clus_Dbscan']]], axis=1, join='inner')

# Check the number of DBSCAN clusters.
num_clus = len(set(labels))
print(set(labels))

{0, 1, 2, -1}


In [226]:
# In order to prevent error, all DBSCAN cluster labels will add 1 to make them non-negative.
bronx_merged_dbscan['Clus_Dbscan'] = bronx_merged_dbscan['Clus_Dbscan'] + 1
# Check the resulting dataframe.
bronx_merged_dbscan.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
0,Bronx,Wakefield,40.894705,-73.847201,2,Ice Cream Shop,Sandwich Place,Gas Station,Caribbean Restaurant,Laundromat,Donut Shop,Pharmacy,Dessert Shop,Discount Store,Distillery,1
1,Bronx,Co-op City,40.874294,-73.829939,1,Bus Station,Accessories Store,Gift Shop,Pharmacy,Park,Discount Store,Restaurant,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant,1
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Bowling Alley,Pizza Place,Platform,Convenience Store,Donut Shop,Chinese Restaurant,1
3,Bronx,Fieldston,40.895437,-73.905643,4,Plaza,Music Venue,Bus Station,River,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,2
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Plaza,Gym,Bus Station,Moving Target,Bank,Food Truck,Home Service,Health & Beauty Service,Donut Shop,2


In [227]:
# Cluster 1
bronx_merged_dbscan.loc[bronx_merged_dbscan['Clus_Dbscan'] == 1, bronx_merged_dbscan.columns[[1] + list(range(5, bronx_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
0,Wakefield,Ice Cream Shop,Sandwich Place,Gas Station,Caribbean Restaurant,Laundromat,Donut Shop,Pharmacy,Dessert Shop,Discount Store,Distillery,1
1,Co-op City,Bus Station,Accessories Store,Gift Shop,Pharmacy,Park,Discount Store,Restaurant,Chinese Restaurant,Salon / Barbershop,Fast Food Restaurant,1
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Bowling Alley,Pizza Place,Platform,Convenience Store,Donut Shop,Chinese Restaurant,1
8,Williamsbridge,Spa,Soup Place,Bar,Nightclub,Caribbean Restaurant,Farmers Market,Food,Flower Shop,Fish Market,Fish & Chips Shop,1
9,Baychester,Donut Shop,Convenience Store,Bank,Pet Store,Discount Store,Sandwich Place,Electronics Store,Fast Food Restaurant,Mexican Restaurant,Men's Store,1
10,Pelham Parkway,Pizza Place,Chinese Restaurant,Italian Restaurant,Home Service,Sushi Restaurant,Plaza,Coffee Shop,Donut Shop,Sandwich Place,Café,1
16,East Tremont,Pizza Place,Shoe Store,Cosmetics Shop,Café,Fast Food Restaurant,Deli / Bodega,Spanish Restaurant,Breakfast Spot,Mobile Phone Shop,Supermarket,1
17,West Farms,Bus Station,Park,Donut Shop,Metro Station,Scenic Lookout,Sandwich Place,Lounge,Supermarket,Basketball Court,Bus Line,1
26,Clason Point,Park,Grocery Store,Boat or Ferry,Bus Stop,South American Restaurant,Pool,Diner,Farmers Market,Flower Shop,Fish Market,1
28,Country Club,Sandwich Place,Playground,Spa,Italian Restaurant,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,1


In [228]:
# Cluster 2
bronx_merged_dbscan.loc[bronx_merged_dbscan['Clus_Dbscan'] == 2, bronx_merged_dbscan.columns[[1] + list(range(5, bronx_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
3,Fieldston,Plaza,Music Venue,Bus Station,River,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,2
4,Riverdale,Park,Plaza,Gym,Bus Station,Moving Target,Bank,Food Truck,Home Service,Health & Beauty Service,Donut Shop,2
5,Kingsbridge,Pizza Place,Sandwich Place,Bar,Supermarket,Latin American Restaurant,Deli / Bodega,Mexican Restaurant,Discount Store,Pharmacy,Spanish Restaurant,2
7,Norwood,Pizza Place,Park,Bank,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Mexican Restaurant,Sandwich Place,Caribbean Restaurant,Spanish Restaurant,2
12,Bedford Park,Diner,Chinese Restaurant,Supermarket,Pizza Place,Bus Station,Deli / Bodega,Sandwich Place,Mexican Restaurant,Spanish Restaurant,Park,2
13,University Heights,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Bakery,Fried Chicken Joint,Convenience Store,Supermarket,Pharmacy,Cosmetics Shop,Donut Shop,2
14,Morris Heights,Pharmacy,Check Cashing Service,Spanish Restaurant,Pizza Place,Recreation Center,Latin American Restaurant,Grocery Store,Food,Bank,Eastern European Restaurant,2
15,Fordham,Mobile Phone Shop,Fast Food Restaurant,Shoe Store,Bank,Donut Shop,Spanish Restaurant,Gym / Fitness Center,Clothing Store,Pizza Place,Supplement Shop,2
18,High Bridge,Pizza Place,Pharmacy,Chinese Restaurant,Deli / Bodega,Fast Food Restaurant,Food,Spanish Restaurant,Food Truck,Supermarket,Market,2
19,Melrose,Pizza Place,Pharmacy,Gym / Fitness Center,Discount Store,Sandwich Place,Supermarket,Supplement Shop,Deli / Bodega,Department Store,Paper / Office Supplies Store,2


In [229]:
# Cluster 3
bronx_merged_dbscan.loc[bronx_merged_dbscan['Clus_Dbscan'] == 3, bronx_merged_dbscan.columns[[1] + list(range(5, bronx_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
27,Throgs Neck,Deli / Bodega,American Restaurant,Pizza Place,Italian Restaurant,Juice Bar,Asian Restaurant,Sports Bar,Coffee Shop,Bar,Electronics Store,3
37,Schuylerville,Bank,Pizza Place,Pharmacy,Mexican Restaurant,Donut Shop,Fast Food Restaurant,Sandwich Place,Supermarket,Sushi Restaurant,Bar,3
38,Edgewater Park,Italian Restaurant,Deli / Bodega,Pizza Place,Coffee Shop,Donut Shop,Juice Bar,Park,Pub,Sports Bar,Liquor Store,3


In [230]:
# Noise
bronx_merged_dbscan.loc[bronx_merged_dbscan['Clus_Dbscan'] == 0, bronx_merged_dbscan.columns[[1] + list(range(5, bronx_merged_dbscan.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Clus_Dbscan
6,Woodlawn,Pub,Deli / Bodega,Pizza Place,Playground,Cosmetics Shop,Supermarket,Food & Drink Shop,Rental Car Location,Bar,Moving Target,0
11,City Island,Harbor / Marina,Seafood Restaurant,Thrift / Vintage Store,Liquor Store,Bar,Diner,Deli / Bodega,Pizza Place,History Museum,Music Venue,0
20,Mott Haven,Spanish Restaurant,Gym,Donut Shop,Bakery,Pizza Place,Metro Station,Discount Store,Fish & Chips Shop,Mobile Phone Shop,Storage Facility,0
21,Port Morris,Spanish Restaurant,Restaurant,Food Truck,Storage Facility,Furniture / Home Store,Music Venue,Latin American Restaurant,Distillery,Donut Shop,Paper / Office Supplies Store,0
25,Soundview,Chinese Restaurant,Grocery Store,Latin American Restaurant,Breakfast Spot,Bus Station,Fried Chicken Joint,Bus Stop,Burger Joint,Pharmacy,Discount Store,0


In [231]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clus)
ys = [i + x + (i*x)**2 for i in range(num_clus)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bronx_merged_dbscan['Latitude'], bronx_merged_dbscan['Longitude'], bronx_merged_dbscan['Neighborhood'], bronx_merged_dbscan['Clus_Dbscan']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## End of clustering analysis.